# Import the data

In [1]:
import pandas as pd
import requests

In [2]:
df=pd.read_csv('P3_GrantExport.csv',sep=';',usecols=[0,6,7,13])

In [3]:
df=df.rename(columns={'\ufeff"Project Number"':'Project','Approved Amount':'Amount'})

In [4]:
df.describe()

,Project
count,63969.000000
mean,84723.419453
std,53406.795178
min,1.000000
25%,36692.000000
50%,101689.000000
75%,133018.000000
max,171414.000000


# Database creation

In [5]:
# Remove missing data
nan_string='data not included in P3'
# Amount NaN
df=df[~df.Amount.isin([nan_string])]
# Institution NaN
df=df[df.Institution.notnull() | (df.University.notnull() & ~df.University.isin(['Nicht zuteilbar - NA']))]
df.shape

(49823, 4)

In [6]:
# Change Amount type
df.Amount=pd.to_numeric(df.Amount)
df.dtypes

Project          int64
Institution     object
University      object
Amount         float64
dtype: object

In [7]:
# Setting index
df=df.set_index('Project')

In [8]:
df.head()

,Institution,University,Amount
Project,,,
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.0
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.0
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0
8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.0


## Extraction of Canton

In [27]:
username='deatinor'
url='http://api.geonames.org/postalCodeSearchJSON?'

In [28]:
correspondencies_dictionary={'Inst. de Hautes Etudes Internat. et du Dév - IHEID': 'GE'}

In [29]:
params={'username':username,'placename':'CH','maxRows':1,'country':'CH','operator':'OR'}
r=requests.get(url,params=params)

In [30]:
i=0
df_final=pd.DataFrame({'Canton':[]})
for university in df.University:
    try:
        canton=correspondencies_dictionary[university]
        df2=pd.DataFrame({'Canton':[canton]})
        df_final=df_final.append(df2)
    except:
        params['placename']=university
        r=requests.get(url,params=params)
        df1=pd.read_json(r.text,orient='records')
        df2=pd.DataFrame(df1.postalCodes[0],columns=['adminCode1'],index=['adminCode1'])
        df2=df2.rename(columns={'adminCode1':'Canton'})
        df_final=df_final.append(df2)
        correspondencies_dictionary[university]=df2.Canton[0]
    i+=1
    if i%5000==0:
        print(i)


2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000


In [32]:
df_final.shape

(49823, 1)

In [33]:
correspondencies_dictionary

{'Inst. de Hautes Etudes Internat. et du Dév - IHEID': 'GE',
 'Robert Walser-Stiftung Bern - RWS': 'BE',
 'Paul Scherrer Institut - PSI': 'AG',
 'Schweizer Paraplegiker Forschung - SPF': 'VD',
 'Forschungsinstitut für biologischen Landbau - FIBL': 'VD',
 nan: 'VD',
 'Université de Fribourg - FR': 'FR',
 'Fachhochschule Ostschweiz - FHO': 'VD',
 'Swiss Institute of Bioinformatics - SIB': 'VD',
 'Zürcher Fachhochschule (ohne PH) - ZFH': 'VD',
 'SUP della Svizzera italiana - SUPSI': 'VD',
 'Staatsunabh. Theologische Hochschule Basel - STHB': 'BS',
 'Institut Universitaire Kurt Bösch - IUKB': 'VD',
 'Kantonsspital St. Gallen - KSPSG': 'SG',
 'Institut für Kulturforschung Graubünden - IKG': 'GR',
 'Interkant. Hochschule für Heilpädagogik ZH - HfH': 'ZH',
 'Eidg. Hochschulinstitut für Berufsbildung - EHB': 'VD',
 'Biotechnologie Institut Thurgau - BITG': 'TG',
 'Berner Fachhochschule - BFH': 'VD',
 'NPO (Biblioth., Museen, Verwalt.) - NPO': 'VD',
 'Pädagogische Hochschule Graubünden - PHGR':

In [41]:
df=df.reset_index()
del df['index']

ValueError: cannot insert level_0, already exists

In [43]:
del df['level_0']
del df['index']

In [59]:
df_final['index']=range(df_final.shape[0])
df_final=df_final.set_index(['index'])
df['Canton']=df_final['Canton']
df

,Project,Institution,University,Amount,Canton
0,4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.0,GE
1,5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0,VD
2,6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.0,BS
3,7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0,VD
4,8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.0,FR
5,9,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,25403.0,FR
6,10,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,47100.0,ZH
7,11,NaN,Université de Lausanne - LA,25814.0,VD
8,13,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE,360000.0,GE
9,14,Klinische Psychologie und Psychotherapie Insti...,Université de Fribourg - FR,153886.0,FR


# Folium visualization

In [60]:
a=df.groupby(df.Canton)

In [66]:
df_group=pd.DataFrame({'Canton':[],'Amount':[]})

In [67]:
for i,group in a:
    print(i)
    s=group['Amount'].sum()
    df_group=df_group.append(pd.DataFrame({'Canton':[i],'Amount':[s]}))

AG
BE
BS
FR
GE
GR
LU
NE
SG
SH
SO
SZ
TG
TI
VD
VS
ZG
ZH


In [72]:
canton_list=['ZH','BE','LU','UR','SZ','OW','NW','GL','ZG','FR','SO','BS','BL','SH','AR','AI','SG','GR','AG','TG','TI','VD','VS','NE','GE','JU']

In [74]:
for i in canton_list:
    isin=df_group.Canton.isin([i]).sum()
    if not isin:
        df_group=df_group.append(pd.DataFrame({'Canton':[i],'Amount':[0]}))

(26, 2)

In [79]:
df_group

,Amount,Canton
0,1.154283e+08,AG
0,1.521933e+09,BE
0,1.352269e+09,BS
0,4.575262e+08,FR
0,1.877102e+09,GE
0,3.542842e+07,GR
0,5.272908e+07,LU
0,3.838320e+08,NE
0,8.675705e+07,SG
0,1.766910e+05,SH


In [90]:
import folium
map_osm = folium.Map(location=[47, 7])
state_geo = r'ch-cantons.topojson.json'
map_osm.choropleth(geo_path=state_geo,key_on='objects.cantons.geometries.id',data=df_group,
                   columns=['Canton','Amount'],fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
                  legend_name='Unemployment Rate (%)',threshold_scale=[5, 6, 7, 8, 9, 10])
map_osm.save('prova.html')

AttributeError: 'NoneType' object has no attribute 'get'